# Introduction to maximum likelihood estimation

## A simple example: Coin tosses

-   Coin tosses have binary outcomes: a Head (H) or a Tail (T).
-   Assume that different coin tosses don’t impact each other.
-   In statistics, this implies that coin toss outcomes are independent
    and identically distributed, or i.i.d..
-   Assume that the coin is biased.
-   Let the probability of getting a Head be $p$ and the probability of
    getting a Tail be `1-p`.
-   So how do we find that value of $p$?

Let’s toss the coin five times, and assume that we get the following
sequence: H,T,T,H,H.

The probability of seeing this result is
$$L(p) = p(1-p)(1-p)pp = p^3(1-p)^2$$ where `3` is the number of Heads
and 2 is the number of Tails. More generally, if we have a total of $N$
tosses, out of which $n$ are Heads, then the probability is written in a
generic function form: $$L(p) = p^n(1-p)^{N-n}$$

-   Here, $L(p)$ is the likelihood of “observing” the data.
-   It is a function of the unknown parameter $p$.
-   Intuitively, it makes sense to use the value, $\hat{p}$, that
    maximizes $L(p)$ to estimate $p$.
-   The maximum likelihood estimator (MLE) is
    $$\hat{p}=\arg\max_{p}L(p).$$

The estimation problem now becomes an optimization problem.

-   Here we can differentiate $L$ with respect to $p$ and set it equal
    to zero to find the optimal value of $p$ $$L'(p)= \frac{dL}{dp}=0$$
-   This will give us this ugly expression:
    $$L'(p)=np^{n-1}(1-p)^{N-n} - (N-n)(1-p)^{N-n-1}p^n=0$$
-   This equation is not easy to solve, both analytically and
    numerically.

Now we consider the log-likelihood function.

-   Let $l(p)=\log\{L(p)\}$, namely,
    $$l(p) = n\log p + (N-n)\log(1-p).$$
-   The maximizer of $L(P)$ is the same as the maximizer of $l(p)$.
-   Thus $\hat{p}$ is the solution to
    $$l'(p)= \frac{dl}{dp}=\frac{n}{p}-\frac{N-n}{1-p}=0,$$ which is
    $$\hat{p}=\frac{n}{N}.$$

``` latex
\begin{euqation}
  l(p) = n\log p + (N-n)log(1-p)
L'(p)=np^{n-1}(1-p)^{N-n} - (N-n)(1-p)^{N-n-1}p^n=0.
\end{euqation}
```

Let’s generate a variable that represents the coin tosses:

``` julia
n = 3
N = 5
p̂̂p̂ = n/N
```

sample statistic population parameter description p̂ p-h

Since we know that the log likelihood (LL) function equals:
`LL(p) = 3 *log(p) + 2 * log(1-p)`, we can also plot this as follows:

``` julia
🍎=1
i = 1
while true
    global i += 1
    if i > 10
        break
    end
end
rand(3)
```

which gives us this figure:

.u .v .it .aj .oc .od .jp .zq width=“700” height=“467”
role=“presentation”}

For the figure we can see that the LL has a maximum point around 0.6. As
we discussed in
<a href="https://medium.com/the-stata-guide/mata-statas-end-game-5983c0ee11bd" class="cc le">the Mata guide</a>,
one of features we can utilize are `optimize` functions. Let’s use them
to find the maximum point:

``` if
mata
void myfunc(todo, x, y, g, H) {
 y = 3 * log(x) + 2 * log(1-x)
 }maxval = optimize_init()
optimize_init_which(maxval, "max")
optimize_init_evaluator(maxval, &myfunc())
optimize_init_params(maxval, 0.2)
xmax = optimize(maxval)xmax
end
```

Without going too much in the details, here we define a function that we
want to maximize, and start the search with an arbitrarily chosen value
of 0.2 which we can see is closer to the maximum point in the figure.
For such a simple optimization problem, the starting value shouldn’t
really matter. From the code above, we get the value of `xmax = 0.6`,
which is what we calculated by hand as well.

We can also plot this on the graph above:

``` if
mata
void myfunc(todo, x, y, g, H) {
 y = 3 * log(x) + 2 * log(1-x)
 }maxval = optimize_init()
optimize_init_which(maxval, "max")
optimize_init_evaluator(maxval, &myfunc())
optimize_init_params(maxval, 0)
xmax = optimize(maxval)xmax
ymax = 3 * log(xmax) + 2 * log(1-xmax)st_local("maxx", strofreal(xmax))
st_local("maxy", strofreal(ymax))
endtwoway (function 3 * log(x) + 2 * log(1-x), range(0 1)), ///
 yline(`maxy') xline(`maxx') ytitle("LL") xlabel(0(0.2)1)
```

from which we get a LL max value of -3.365:

.u .v .it .aj .oc .od .jp .zq width=“700” height=“467”
role=“presentation”}

But since Stata has built-in MLE functions (see `help ml`) with fairly
extensive documentation, we don’t need to jump through the hoops to
define an optimization problem in Mata. We can simply use the `ml`
function. If you see the documentation of this command, it won’t be
surprising if you are immediately confused. This use of this command is
a bit convoluted and takes some time to get used to the various options.
In order to use `ml` commands, we need to write small programs where we
define the LL functions that need to be optimized. In the coin toss
case, since we already know the LL function which is linear in form, we
can make use of the `mlexp` option (see `help mlexp`). The documentation
of `mlexp` says the following:

.u .v .it .aj .oc .od .jp .zq width=“573” height=“226”
role=“presentation”}

So here, since we know the LL which fulfills the conditions defined
above, we making use of the special `mlexp` option. The `ml` is more
generic, but we will get to it later. We can express our function as
follows:

``` if
mlexp ((x * log({p})) + ((1 - x) * log(1 - {p})))
```

and that is all we need. A single line equation. There are of course a
host of various other options that go with it including defining
initialization of parameters, other constraints etc. Note in the command
above that `x` is the Stata variable that we have generated, and `{p}`
is the value we need to maximize. This value should always be written in
curly brackets. There can also be more than one parameter, each in its
own curly brackets. From the expression above, we get this output:

.u .v .it .aj .oc .od .jp .zq width=“687” height=“347”
role=“presentation”}

The iteration values are similar to what the optimize gives us. In fact,
here I would like to point out that in the background, the `ml`
functions actually use the Mata `optimize` routines. So now you know
where all the magic happens.

The value of $p$ is correctly estimated as `0.6`. The standard error is
derived from the asymptotic variance of the finite sample, which in our
case is very small. If we increase the number of coin tosses, then $p$
{.mq .mr .ms .mt .mu .b} should approach 0.5, and the standard error
should approach 0, the true values of the population distribution. We
can check this as follows:

``` if
clear all
set obs 1000000  // or higher or lowergen x = uniform() < .5  // a 0/1 variablemlexp ((x * log({p})) + ((1 - x) * log(1 - {p})))
```

which gives us:

.u .v .it .aj .oc .od .jp .zq width=“700” height=“436”
role=“presentation”}

The command `mlexp` also has several post estimation options for
conducting various tests, linear combination of parameters, creating
margins plot etc. See `help mlexp postestimation` for more details. See
also
<a href="https://blog.stata.com/2016/06/14/multiple-equation-models-estimation-and-marginal-effects-using-mlexp/" class="cc le">this Stata blog entry</a>
for a more advanced application of `mlexp`.

Now on to more standard applications.

# Logit regressions {#2366 .mv .mw .ds .aw .ax .mx .my .hl .mz .na .nb .hp .nc .nd .ne .nf .ng .nh .ni .nj .nk .nl .nm .nn .no .np .ep selectable-paragraph=““}

Another common example of MLEs are Logit or Probit regressions. These
are standard tools when dealing with dichotomous outcomes that can be
explained by some independent set of variables. For binary outcomes, a
linear regression is obviously a bad choice here since it will predict
outcomes that are fall outside the plausible \[0,1\] range. This is
where the S-shaped Logit and Probit functions are really handy since
they contain the outcomes within this range. From these models, we
basically recover the *probability* of having a value of one. Again this
theory you should either know or read up on.

The likelihood of the Logit is derived from a logistic distribution,
which takes the following functional form:

.u .v .it .aj .oc .od .jp .zq width=“478” height=“127”
role=“presentation”}

While this formula might seem complex, it follows the same logic as the
coin toss example. Here we have have a probability if `y = 1`:

``` if
Pr(y = 1) = exp(z) / (1 + exp(z))
```

and a probability for `y = 0`:

``` if
Pr(y = 0) = 1 - exp(z) / (1 + exp(z))
          = 1 / (1 + exp(z))
```

The likelihood formula shown above is the joint probability distribution
of the binary outcome variable *y*.

The likelihood of the Probit model is derived from a normal
distribution, that looks like this expression:

.u .v .it .aj .oc .od .jp .zq width=“700” height=“124”
role=“presentation”}

In both the likelihood distributions, `z = X beta` is a matrix of
explanatory variables and `n` are the number of observations going from
1 to `N`.

Since these formulas are a pain to solve “by hand” especially the Probit
model, I will stick to the Logit since it is the easier one to deal
with. Like the coin toss example, I will use a custom set of values to
keep this relatively tractable. Let’s generate five observations:

``` if
clear allset obs 5gen y = .
gen x = .replace y = 1 in 1
replace y = 0 in 2
replace y = 0 in 3
replace y = 1 in 4
replace y = 1 in 5replace x = -1 in 1
replace x =  1 in 2
replace x = -4 in 3
replace x =  8 in 4
replace x = -5 in 5
```

where `y` is the outcome variable, and `x` is the explanatory variable.
There can be many more explanatory variables but since we want to graph
the likelihood functions, we just use a single explanatory variable. In
our model, we also include an intercept, such that `y` is explained as a
logistic function of the `z = alpha + beta x`.

For the observations above, the likelihood function can be derived as:

.u .v .it .aj .oc .od .jp .zq width=“682” height=“100”
role=“presentation”}

Since this is a pain to differentiate and solve for `z={alpha, beta}`,
we therefore make use of the log likelihood (LL) expression:

.u .v .it .aj .oc .od .jp .zq width=“700” height=“71”
role=“presentation”}

which seems a bit more benign than the likelihood. The generic logit LL
can be written as:

.u .v .it .aj .oc .od .jp .zq width=“512” height=“86”
role=“presentation”}

Since we have the LL explain by two parameters `alpha` and `beta`, we
can also visualize this as follows:

<figure>
<figcaption aria-hidden="true">Figure generated in Mathematica 12.1</figcaption>
</figure>

where we can see that the LL has a maximum point somewhere in the red
zone.

In order to solve this in Stata, we make use of the standard `ml`
functions (see `help ml`). For the logit case, we need define a program
as follows:

``` if
cap prog drop mllogitprogram define mllogit
     args lnf Xb
     quietly replace `lnf' = ln(    invlogit(`Xb')) if $ML_y1==1
     quietly replace `lnf' = ln(1 - invlogit(`Xb')) if $ML_y1==0
end
```

Based on the code above, the `ml` arguments are described as follows.
The program requires arguments `args` where we provide two `lnf` and
`Xb`. These could be any names but are suggested in Stata documentation
for clarity. The first `lnf` is an abbreviation for “linear form” since
we have a linear LL model here. The second one `Xb` is the generic term
for covariates. We specify the conditions for the dependent variable
*y=0* and *y=1* separately. Here *y* is denoted by the generic term
`$ML_y1`. The distribution `invlogit` is a built-in function where
`invlogit(x) = exp(x)/(1 + exp(x))`. An alternative way to specify the
the above conditions would have been:

``` if
program define mllogit2
     args lnf Xbquietly replace `lnf' =       - ln(1+exp(-`Xb')) if $ML_y1==1
quietly replace `lnf' = -`Xb' - ln(1+exp(-`Xb')) if $ML_y1==0end
```

which is a bit more verbose, but this is up to taste. More advanced
programmers would save the program in a separate ado file which is
called in a do file. But for now, we keep this program itself inside the
dofile to avoid complicating the whole thing further. Note for
optimization, it does not matter if you use `log` or `ln`. It won’t
change the results.

Next step we need to call this program in the `ml` instance. This is
done as follows:

``` if
ml model lf mllogit (y = x)
```

Here **lf** is the evaluator for the linear form. There are several
other evaluators built inside Stata, which are used for the `optimize`
functions as well. I won’t go in their details here since it requires
introducing some more theory, but basically `lf` corresponds to `lf0`
(the default) which estimates the parameters, `lf1` estimates the
parameters plus the gradients (first derivative), and `lf2` calculates
the parameters, gradients, and the Hessian (second derivative). These
require additional inputs in the `ml` functions and are used for more
complex calculations especially if the functions are highly non-linear
or some custom initializations are required, or if there are
restrictions on the gradients and the Hessians.

After we use the above `ml` command, a bunch of options open up. You can
try these and also look up their documentations:

``` if
ml check  // conducts tests to see if the ML is properly set up
ml query  // a complex summary of the problem set upml search // searches for decent initial values
ml plot x // same as search but uses a graphical interface
ml graph  // graphs the log likelihood value and the iteration
```

In order to actually solve the model, we can write:

``` if
ml maximize  // or minimize depending on the setup
```

which gives us this output:

.u .v .it .aj .oc .od .jp .zq width=“672” height=“350”
role=“presentation”}

where we get the intercept (`alpha`) and the slope of x variable
(`beta`). We can also check this against the standard logit command:

``` if
logit y x
```

.u .v .it .aj .oc .od .jp .zq width=“634” height=“336”
role=“presentation”}

which also gives the same output but with fewer iterations. Here the
`logit` (or the `logistic`, both commands are same in Stata), also
utilizes the `ml` option in the background to derive these results.

Just as an additional point, and since several people struggle with
this, we can also draw the logistic curve as follows:

``` if
logit y xmat li e(b)global b0 = e(b)[1,2]
global b1 = e(b)[1,1]display "($b0 * x) + $b1"cap drop yhat
predict yhattwoway  /// 
 (scatter yhat x) ///
 (function exp(($b1 * x) + $b0) / (1 +exp(($b1 * x) + $b0) ), range(-60 60)) ///
 , ytitle("Probability") legend(order(1 "Fitted values" 2 "Fitted curve"))
```

which gives us this figure:

.u .v .it .aj .oc .od .jp .zq width=“700” height=“467”
role=“presentation”}

With logit functions, one can also derive margin plots, odds ratios,
etc. but these additional post-estimation calculations we won’t touch
here. These can be extracted from the derived estimates and it is useful
to refer to the Stata manual or some text books.

# Probit functions

Instead of the logit function, we can also use the Probit function
derived from the normal distribution. The LL for the Probit is simply
the log of the likelihood we defined earlier. Since the actual LL for
our data looks fairly ugly (there won’t be enough space to even fit it
in one or two lines), I won’t post it here but this sort of stuff is
easy to derive symbolically in Mathematica.

Just out of curiosity, if we compare the LL functions of the Probit and
the logit, we can see the differences clearly:

!\[Logit LL is flatter than the Profit LL. Figure generated in
Mathematica .u .v .it .aj .oc .od .jp .zq width=“700” height=“477”
role=“presentation”}

The flatter surface is the Logit LL while the more concave shape is the
Probit LL. Here you can also see that the edges of the Probit function
starts getting a bit rough. These are most likely out-of-sample domains
where one starts hitting some non-linear combination of the variables.

In Stata, we can define the LL for the Probit as follows:

``` if
cap prog drop mlprobitprogram mlprobit
 args lnf xb
   quietly replace `lnf' = ln(    normal(`xb')) if $ML_y1==1
   quietly replace `lnf' = ln(1 - normal(`xb')) if $ML_y1==0
endml model lf mlprobit (y = x)
ml maximize
```

which gives us:

.u .v .it .aj .oc .od .jp .zq width=“691” height=“591”
role=“presentation”}

This can also be compared with the standard Stata command:

``` if
probit y x
```

and to draw the graphs that compare the Probit with the Logit, we can
use the following set of commands:

``` if
probit y xglobal p0 = e(b)[1,2]
global p1 = e(b)[1,1]twoway  /// 
 (function y = exp(($b1 * x) + $b0) / (1 +exp(($b1 * x) + $b0) ), range(-60 60)) ///
 (function y = normal(($p1 * x) + $p0), range(-60 60)) ///
 , ytitle("Probability") ///
 legend(order(1 "Logit" 2 "Probit"))
```

which gives us this graph:

.u .v .it .aj .oc .od .jp .zq width=“700” height=“467”
role=“presentation”}

Here we can see differences in the cumulative density functions (CDFs)
for the two functions.

## Using actual data {#8459 .op .mw .ds .aw .ax .qe .qf .qg .mz .qh .qi .qj .nc .qk .ql .qm .ng .qn .qo .qp .nk .qq .qr .qs .no .qt .ep selectable-paragraph=““}

Let’s try what we learned on actual data. Here we will use Stata’s
*union* dataset, that is also frequently used in their help files for
examples with binary outcomes.

``` if
webuse union, clear
```

Here we use a simple specification, where we explain *union* membership
(a binary outcome) as a function of *age*, *grade* (education
completed), (living in the) *south*, and (being) *black*.

We can run a simple `logit` and `probit` in Stata as follows:

``` if
logit union age grade south black
probit union age grade south black
```

In ML form we can use the functions we defined above:

``` if
cap prog drop mllogit
program define mllogit
     args lnf Xb
     quietly replace `lnf' = ln(    invlogit(`Xb')) if $ML_y1==1
     quietly replace `lnf' = ln(1 - invlogit(`Xb')) if $ML_y1==0
endcap prog drop mlprobit
program mlprobit
 args lnf xb
    quietly replace `lnf' = ln(    normal(`xb')) if $ML_y1==1
    quietly replace `lnf' = ln(1 - normal(`xb')) if $ML_y1==0
endml model lf mllogit (union = age grade south black)
ml maximizeml model lf mlprobit (union = age grade south black)
ml maximize
```

The code above will spit out the parameter values that maximize the LL
function. You can also check the results against the standard commands.
As mentioned earlier, the Stata `probit` and `logit` commands utilize
the `ml` functions, which themselves uses the Mata `optimize`. The
inter-connectedness of these routines also helps give a picture of how
various different functions are built on top of each other. And so
running `logit y x1 x2…` seems like a fairly easy command to use, there
is a lot happening in the background which makes it easy to use!

# Other ML functions? {#9ea5 .mv .mw .ds .aw .ax .mx .my .hl .mz .na .nb .hp .nc .nd .ne .nf .ng .nh .ni .nj .nk .nl .nm .nn .no .np .ep selectable-paragraph=““}

Since there are tons of distributions that one can apply to various data
problems, there are tons of MLE functions as well. This
<a href="https://stats.idre.ucla.edu/stata/code/imple-linear-and-nonlinear-models-using-statas-ml-command/" class="cc le">UCLA website</a>
lists a bunch of them. But one I would like to highlight here is the MLE
to deal with standard regressions. Using the auto dataset, this takes
the following form:

``` if
sysuse auto, clearcapture program drop myols
program myols
  
args lnf Xb lnsigma
local y "$ML_y1"
quietly replace `lnf' = ln(normalden(`y', `Xb', exp(`lnsigma')))
endml model lf myols (xb: mpg = weight foreign) (lnsigma:)
ml maximize
display exp([lnsigma]_cons)
```

which gives this output:

.u .v .it .aj .oc .od .jp .zq width=“700” height=“704”
role=“presentation”}

where the `lnsigma` term is the mean squared error (MSE) of the
regression. Now if we compare it with the standard OLS command:

``` if
regress mpg weight foreign
```

we get this output:

.u .v .it .aj .oc .od .jp .zq width=“606” height=“296”
role=“presentation”}

Note here that the coefficients are exactly the same but standard errors
and the MSE terms are different. Here, unlike `logit` and `probit`,
which also utilize `optimize` and hence results are similar, for OLS the
estimations routes are very different. The `regress` command use
standard matrix algebra while the `ml` option above is utilizing
asymptotic properties of the estimators. Even though the differences are
very small, the `ml` results should be considered better (or BLUEr) than
the OLS estimates.

There are still several things not covered in this guide but are worth
looking up if you are serious about learning `ml`. These include more
complex functional forms, constraints on MLE functions, optimization
techniques (Newton-Rhapson, Berndt-Hall-Hall-Hausman etc.), search
bounds etc. Currently there is only one book by Gould, Pitblado, and Poi
which covers this topic in detail: